# Notebook to scrape data about Law School faculty members

Requirements:
    
    !pip install pandas
    !pip install bs4
    !pip install requests

Currently scrapes:

    (1) Osgoode Hall Law School, York University (osgoode_bios.json)
    (2) Faculty of Law, University of Toronto (u_toronto_bios.json)
    (3) Lincoln Alexander School of Law, Toronto Metropolitan University (tmu_bios_json)
    
License: [CC BY-NC 3.0](https://creativecommons.org/licenses/by-nc/3.0/)

### Setup

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

# set paths
json_outpath = 'data/'

### (1) Scrape Osgoode website

In [2]:
# Get all links for individual faculty members webpages

# load main faculty page
url = 'https://www.osgoode.yorku.ca/faculty/'
r = requests.get(url)
soup = BeautifulSoup(r.content, 'html.parser')

# get the main table on faculty page, convert to dataframe and clean
tables = soup.find_all('table')
df = pd.read_html(str(tables[0]))[0]
df.columns = df.columns.str.lower()
df.drop('unnamed: 0', axis=1, inplace=True)

# get all the links (hrefs) on faculty page
results = []
links = soup.find_all('a')
for link in links:
    if 'https://www.osgoode.yorku.ca/faculty-and-staff/' in link.get('href'):
        results.append(link.get('href'))

# delete every second link (b/c the links are duplicated)
results = results[::2]

# add links to the dataframe
df['href'] = results

df

,name,title,email,telephone,office,href
0,Rabiat Akande,Assistant Professor,rakande@osgoode.yorku.ca,416-650-8422,3048,https://www.osgoode.yorku.ca/faculty-and-staff...
1,Harry Arthurs,Professor Emeritus,harthurs@osgoode.yorku.ca,NaN,3015,https://www.osgoode.yorku.ca/faculty-and-staff...
2,Saptarishi Bandopadhyay,Associate Professor,sbandopadhyay@osgoode.yorku.ca,416-736-5488,4053,https://www.osgoode.yorku.ca/faculty-and-staff...
3,Stephanie Ben-Ishai,Professor and York University Distinguished Re...,sbenishai@osgoode.yorku.ca,416-650-8239,3043,https://www.osgoode.yorku.ca/faculty-and-staff...
4,Benjamin L. Berger,Professor & York Research Chair in Pluralism a...,bberger@osgoode.yorku.ca,416-736-5867,3030,https://www.osgoode.yorku.ca/faculty-and-staff...
...,...,...,...,...,...,...
73,Emily Kidd White,Assistant Professor,ekwhite@osgoode.yorku.ca,416-736-5826,3033,https://www.osgoode.yorku.ca/faculty-and-staff...
74,J. Scott Wilkie,Distinguished Professor of Practice,swilkie@osgoode.yorku.ca,416-736-2100 ext. 22189,4065,https://www.osgoode.yorku.ca/faculty-and-staff...
75,Cynthia Williams,Professor Emeritus,cwilliams@osgoode.yorku.ca,416-736-5545,4021,https://www.osgoode.yorku.ca/faculty-and-staff...
76,Alan N. Young,Professor Emeritus,ayoung@osgoode.yorku.ca,NaN,3015,https://www.osgoode.yorku.ca/faculty-and-staff...


In [3]:
# Scrape bios from individual faculty member webpages

# function to parse faculty member page
def parse_faculty_page(url):

    # load faculty member page
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')

    # get text from .entry-content tag
    bio = soup.find('div', {'class': 'entry-content'}).text
    bio = bio.replace('\xa0', ' ') # Remove non-breaking spaces
    bio = ' '.join(bio.split()) # Remove extra whitespace
    bio = bio.strip() # Remove leading and trailing whitespace

    # slow down process to avoid overloading server
    time.sleep(.5)
  
    return bio

# apply function to each row in dataframe
df['bio'] = df['href'].apply(parse_faculty_page) 

# get cleaned bios
def clean_bio(bio):
    bio = bio.split('Graduate Research Supervision (LLM, PhD):')[0]   # remove everything after 'Graduate Research Supervision (LLM, PhD):'
    bio = bio.replace('\n', ' ').strip()
    return bio

df['bio'] = df['bio'].apply(clean_bio)

df['faculty'] = 'osgoode'

# reorder columns & drop unnecessary columns
df = df[['faculty', 'name', 'title', 'email', 'href', 'bio']]


# Save to json for future use
df.to_json(json_outpath+'osgoode_bios.json', orient='records', indent = 2)

df


,faculty,name,title,email,href,bio
0,osgoode,Rabiat Akande,Assistant Professor,rakande@osgoode.yorku.ca,https://www.osgoode.yorku.ca/faculty-and-staff...,Professor Rabiat Akande works in the fields of...
1,osgoode,Harry Arthurs,Professor Emeritus,harthurs@osgoode.yorku.ca,https://www.osgoode.yorku.ca/faculty-and-staff...,"University Professor, former Dean of Osgoode H..."
2,osgoode,Saptarishi Bandopadhyay,Associate Professor,sbandopadhyay@osgoode.yorku.ca,https://www.osgoode.yorku.ca/faculty-and-staff...,I am an Associate Professor at Osgoode Hall La...
3,osgoode,Stephanie Ben-Ishai,Professor and York University Distinguished Re...,sbenishai@osgoode.yorku.ca,https://www.osgoode.yorku.ca/faculty-and-staff...,Professor Stephanie Ben-Ishai is a Distinguish...
4,osgoode,Benjamin L. Berger,Professor & York Research Chair in Pluralism a...,bberger@osgoode.yorku.ca,https://www.osgoode.yorku.ca/faculty-and-staff...,Professor Benjamin L. Berger is Professor and ...
...,...,...,...,...,...,...
73,osgoode,Emily Kidd White,Assistant Professor,ekwhite@osgoode.yorku.ca,https://www.osgoode.yorku.ca/faculty-and-staff...,Professor Emily Kidd White’s areas of teaching...
74,osgoode,J. Scott Wilkie,Distinguished Professor of Practice,swilkie@osgoode.yorku.ca,https://www.osgoode.yorku.ca/faculty-and-staff...,"J. Scott Wilkie, a partner who practises in th..."
75,osgoode,Cynthia Williams,Professor Emeritus,cwilliams@osgoode.yorku.ca,https://www.osgoode.yorku.ca/faculty-and-staff...,Professor Cynthia Williams joined Osgoode Hall...
76,osgoode,Alan N. Young,Professor Emeritus,ayoung@osgoode.yorku.ca,https://www.osgoode.yorku.ca/faculty-and-staff...,Alan Young is the Co-Founder and former Direct...


### (2) Scrape University of Toronto Website

In [4]:
# Define the base url for the faculty website
base_url = "https://www.law.utoronto.ca"

# Get the html content of the faculty page
response = requests.get(base_url+ '/faculty-staff/full-time-faculty')
soup = BeautifulSoup(response.content, "html.parser")

# Find all tables in soup
tables = soup.find_all("table")

# Iterate through all tables, getting 'name', 'phone' and 'email' for each faculty member
results = []
for table in tables:
    result = {}
    rows = table.find_all("tr")
    for row in rows:
        cols = row.find_all("td")
        if len(cols) > 0:
            name = cols[0].text.strip()
            phone = cols[1].text.strip()
            email = cols[2].text.strip()
            href = cols[0].find("a").get("href")
            result = {'name': name, 'email': email, 'href': base_url + href}
            results.append(result)

# convert result to df
df = pd.DataFrame(results)

df

,name,email,href
0,"Aidid, Abdi",a.aidid@utoronto.ca,https://www.law.utoronto.ca/faculty-staff/full...
1,"Alarie, Benjamin",ben.alarie@utoronto.ca,https://www.law.utoronto.ca/faculty-staff/full...
2,"Anand, Anita",website.law@utoronto.ca,https://www.law.utoronto.ca/faculty-staff/full...
3,"Austin, Lisa",lisa.austin@utoronto.ca,https://www.law.utoronto.ca/faculty-staff/full...
4,"Bédard-Rubin, Jean-Christophe",jc.bedardrubin@utoronto.ca,https://www.law.utoronto.ca/faculty-staff/full...
...,...,...,...
57,"Valcke, Catherine",c.valcke@utoronto.ca,https://www.law.utoronto.ca/faculty-staff/full...
58,"Valverde, Mariana",m.valverde@utoronto.ca,https://www.law.utoronto.ca/faculty-staff/full...
59,"Waddams, Stephen",s.waddams@utoronto.ca,https://www.law.utoronto.ca/faculty-staff/full...
60,"Weinrib, Arnold",arnold.weinrib@utoronto.ca,https://www.law.utoronto.ca/faculty-staff/full...


In [5]:
# get faculty bios from each faculty member's page

def get_bio(row):
    bio = []
    response = requests.get(row['href'])
    soup = BeautifulSoup(response.content, "html.parser")

    texts = soup.find_all("div", class_="field")
    for text in texts:
        bio.append(text.text.strip())
    
    bio = '\n'.join(bio)

    time.sleep(0.5)
    return bio

df['bio'] = df.apply(get_bio, axis=1)

# get research areas
def get_research_areas(row):
    response = requests.get(row['href'])
    soup = BeautifulSoup(response.content, "html.parser")
    research_areas = soup.find("div", class_="bottom-right")
    if research_areas:
        research_areas = research_areas.get_text(separator= ',') # use this to prevent words from being concatenated
    else:
        return None
    if 'Research areas,' in research_areas:
        research_areas = research_areas.split('Research areas,')[1]
    elif 'Research Areas,' in research_areas:
        research_areas = research_areas.split('Research Areas,')[1]
    else:
        return None
    research_areas = research_areas.split('\n')[0]
    if research_areas[-1] == ',':
        research_areas = research_areas[:-1]
    research_areas = research_areas.split(',')
    research_areas = ', '.join(research_areas)
    time.sleep(0.5)
    return research_areas

df['research_areas'] = df.apply(get_research_areas, axis=1)

# get cleaned bios
def clean_bio(bio):
    bio = bio.split('\nEducation')[0]
    bio = bio.split('\nSelected Publications')[0]
    bio = bio.split('\nSelected publications')[0]
    bio = bio.split('\nSee also Professor')[0]
    bio = bio.replace('\xa0', ' ')
    bio = bio.replace('\n', ' ').strip()
    return bio

df['bio'] = df['bio'].apply(clean_bio)

# append research areas if any
def append_research_areas(row):
    if row['research_areas']:
        return row['bio'] + ' Research Interests: ' + row['research_areas']
    else:
        return row['bio']

df['bio'] = df.apply(append_research_areas, axis=1)

# add faculty name
df['faculty'] = 'u_toronto'

# revise names to go from Last, first to First Last
def clean_name(name):
    name = name.split(', ')
    name = name[1] + ' ' + name[0]
    return name

df['name'] = df['name'].apply(clean_name)

# create new column for title, with NaN values
df['title'] = None

# reorder columns & drop unnecessary columns
df = df[['faculty', 'name', 'title', 'email', 'href', 'bio']]

# Save to json for future use
df.to_json(json_outpath + 'u_toronto_bios.json', orient='records', indent = 2)

df

,faculty,name,title,email,href,bio
0,u_toronto,Abdi Aidid,None,a.aidid@utoronto.ca,https://www.law.utoronto.ca/faculty-staff/full...,Assistant Professor Jackman Law BuildingRoom J...
1,u_toronto,Benjamin Alarie,None,ben.alarie@utoronto.ca,https://www.law.utoronto.ca/faculty-staff/full...,Professor & Osler Chair in Business Law Jackma...
2,u_toronto,Anita Anand,None,website.law@utoronto.ca,https://www.law.utoronto.ca/faculty-staff/full...,J.R. Kimber Chair in Investor Protection and C...
3,u_toronto,Lisa Austin,None,lisa.austin@utoronto.ca,https://www.law.utoronto.ca/faculty-staff/full...,"Professor, Chair in Law and Technology Jackman..."
4,u_toronto,Jean-Christophe Bédard-Rubin,None,jc.bedardrubin@utoronto.ca,https://www.law.utoronto.ca/faculty-staff/full...,Assistant Professor 78 Queen's Park Assistant ...
...,...,...,...,...,...,...
57,u_toronto,Catherine Valcke,None,c.valcke@utoronto.ca,https://www.law.utoronto.ca/faculty-staff/full...,Professor Jackman Law BuildingRoom J42278 Quee...
58,u_toronto,Mariana Valverde,None,m.valverde@utoronto.ca,https://www.law.utoronto.ca/faculty-staff/full...,Professor of Criminology Complete profile page...
59,u_toronto,Stephen Waddams,None,s.waddams@utoronto.ca,https://www.law.utoronto.ca/faculty-staff/full...,Professor Jackman Law Building78 Queen's ParkT...
60,u_toronto,Arnold Weinrib,None,arnold.weinrib@utoronto.ca,https://www.law.utoronto.ca/faculty-staff/full...,Professor (Retired) Jackman Law Building78 Que...


### (3) Scrape Toronto Metropolitan University website

In [6]:
# Get api data for faculty

url = 'https://www.torontomu.ca/law/faculty-and-research/faculty/jcr:content/content/resbiographystack.data.1.json'
response = requests.get(url)
json_data = response.json()
df = pd.DataFrame(json_data['data'])

# Get api data for cross-appointed faculty
url = 'https://www.torontomu.ca/law/faculty-and-research/faculty/jcr:content/content/resbiographystack_1397589177.data.1.json'
response = requests.get(url)
json_data = response.json()
df2 = pd.DataFrame(json_data['data'])

# Combine faculty data
df = pd.concat([df, df2], ignore_index=True)

df['page']=df['page'].str.replace('/content/ryerson/','https://www.torontomu.ca/')

df

,page,firstname,lastname,title,email,department,specialization,thumbnailImage,thumbnailAltText,tags
0,https://www.torontomu.ca/law/faculty-and-resea...,Idil,Atak,Associate Professor,idil.atak@torontomu.ca,Lincoln Alexander School of Law,"Irregular migration, refugee protection, secur...",/content/dam/law/faculty/Idil_Atak_1200x900.jpg,Idil Atak,NaN
1,https://www.torontomu.ca/law/faculty-and-resea...,Ed,Béchard-Torres,Assistant Professor,NaN,Lincoln Alexander School of Law,Corporate law; contract law; constitutional la...,/content/dam/law/faculty/Ed-Bechard-Torres-120...,Ed Béchard-Torres,NaN
2,https://www.torontomu.ca/law/faculty-and-resea...,Hilary Evans,Cameron,Assistant Professor,h.evanscameron@torontomu.ca,Lincoln Alexander School of Law,Refugee law; administrative law; memory; risk ...,/content/dam/law/faculty/hilary-evans-cameron.jpg,Hilary Evans Cameron,NaN
3,https://www.torontomu.ca/law/faculty-and-resea...,Christopher,Campbell-Duruflé,Assistant Professor,NaN,Lincoln Alexander School of Law,"International law, environmental law, human ri...",/content/dam/law/faculty/Christopher_Campbell-...,Christopher Campbell-Duruflé,"[{'tagId': 'Positions-Titles:Professor', 'tagT..."
4,https://www.torontomu.ca/law/faculty-and-resea...,Scott,Franks,Assistant Professor,scott.franks@torontomu.ca,Lincoln Alexander School of Law,Aboriginal Law; Indigenous legal orders; criti...,/content/dam/law/faculty/Scott_Franks_1200x900...,Scott Franks,NaN
5,https://www.torontomu.ca/law/faculty-and-resea...,Sari,Graben,Associate Dean Research &amp; Graduate Studies...,sgraben@torontomu.ca,Lincoln Alexander School of Law,"Environmental law, Aboriginal Law, Gender, Res...",/content/dam/law/images/sari-graben-new.jpg,Sari Graben,NaN
6,https://www.torontomu.ca/law/faculty-and-resea...,Kathleen (Katie),Hammond,Assistant Professor,kathleen.hammond@torontomu.ca,Lincoln Alexander School of Law,Science and technology law; health law; family...,/content/dam/law/faculty/kathleen-hammond.jpg,Kathleen Hammond,NaN
7,https://www.torontomu.ca/law/faculty-and-resea...,Graham,Hudson,"Associate Dean, Academic; Professor",graham.hudson@torontomu.ca,Lincoln Alexander School of Law,"Socio-legal studies, access to justice, crimin...",/content/dam/law/faculty/Graham_Hudson_new.jpg,"Graham Hudson, Associate Professor",NaN
8,https://www.torontomu.ca/law/faculty-and-resea...,Angela,Lee,Assistant Professor,angela@torontomu.ca,Lincoln Alexander School of Law,Law and technology; food and agriculture law; ...,/content/dam/law/faculty/angela-lee.jpg,Angela Lee,NaN
9,https://www.torontomu.ca/law/faculty-and-resea...,Avner,Levin,Professor,avner.levin@torontomu.ca,Lincoln Alexander School of Law and the busine...,Legal regulation and protection of privacy and...,/content/dam/law/faculty/avner-levin.jpg,Avner Levin,NaN


In [7]:
# Get faculty bios from faculty pages

def get_bio(page):
    response = requests.get(page)
    soup = BeautifulSoup(response.text, 'html.parser')

    # get text from the first div that includes .resText
    bio = soup.find('div', {'class': 'resText'}).text

    # remove newlines
    bio = bio.replace('\n', ' ')

    # remove multiple spaces
    bio = ' '.join(bio.split())

    bio = bio.strip()

    time.sleep(0.5)

    return bio

df['bio'] = df['page'].apply(get_bio)

# rename page
df = df.rename(columns={'page': 'href'})

# combine first and last name
df['name'] = df['firstname'] + ' ' + df['lastname']

# Append specializations to bio
def append_specialization(row):
    bio = row['bio']
    specialization = row['specialization']
    if specialization:
        bio = bio + ' Research Interests: ' + specialization
    return bio

df['bio'] = df.apply(append_specialization, axis=1)

df['faculty'] = 'tmu'

# reorder columns & drop unnecessary columns
df = df[['faculty', 'name', 'title', 'email', 'href', 'bio']]

# Manually add the dean
faculty = 'tmu'
name = 'Donna Young'
title = 'Dean'
email = 'deanoflaw@torontomu.ca'
hfref = 'https://www.torontomu.ca/law/about/our-dean/'
bio = 'Donna E. Young is the Founding Dean of the Lincoln Alexander School of Law. Before assuming her deanship, she was the President William McKinley Distinguished Professor of Law and Public Policy at Albany Law School and a joint faculty member at the University at Albany\'s Department of Women\'s, Gender, and Sexuality Studies. Her teaching and scholarship focus on law and inequality, race and gender discrimination, and academic freedom and university governance. She has taught courses in Criminal Law, Employment Law; U.S. Federal Civil Procedure; Gender and Work; and Race, Rape Culture, and Law. Dean Young is much sought after as a speaker and has been invited to present her work at conferences and other venues around the world. She has been a staff member at the American Association of University Professors\' (AAUP) Department of Academic Freedom, Tenure, and Governance, in Washington, D.C. and was a member of the AAUP\'s Committee A, the preeminent national body setting standards and investigating academic freedom disputes in the United States. She has been a Fellow at Cornell Law School\'s Gender, Sexuality, and Family Project; a Visiting Scholar at Osgoode Hall Law School\'s Institute of Feminist Legal Studies; an Associate in Law at Columbia Law School; a Visiting Scholar at the Faculty of Law at Roma Tre University in Rome, Italy; and a consultant to the International Development Law Organization for whom she traveled to Uganda to conduct field research on the relationship between gender inequality and law in the context of the HIV/AIDS crisis. Dean Young\'s previous professional experiences include articling at Cornish Roland - a labour law firm in Toronto; serving as a consultant with the Ontario Human Rights Commission; and working as a researcher with the NYC Office of Labor Relations. She is admitted to practice in New York State. Research interests: Criminal Law; Employment Law; US Federal Civil Procedure; Antidiscrimination Law and Civil Rights; Critical Race Theory and Feminist Legal Theory; Academic freedom and due process, and university governance; Title IX'

df2 = pd.DataFrame([[faculty, name, title, email, hfref, bio]], columns=['faculty', 'name', 'title', 'email', 'href', 'bio'])
df = pd.concat([df, df2], ignore_index=True)



# Save to json for future use
df.to_json(json_outpath + 'tmu_bios.json', orient='records', indent = 2)

df

,faculty,name,title,email,href,bio
0,tmu,Idil Atak,Associate Professor,idil.atak@torontomu.ca,https://www.torontomu.ca/law/faculty-and-resea...,Dr. Idil Atak is an associate professor at the...
1,tmu,Ed Béchard-Torres,Assistant Professor,NaN,https://www.torontomu.ca/law/faculty-and-resea...,Ed Béchard-Torres is an assistant professor in...
2,tmu,Hilary Evans Cameron,Assistant Professor,h.evanscameron@torontomu.ca,https://www.torontomu.ca/law/faculty-and-resea...,"A former litigator, Hilary Evans Cameron repre..."
3,tmu,Christopher Campbell-Duruflé,Assistant Professor,NaN,https://www.torontomu.ca/law/faculty-and-resea...,Christopher Campbell-Duruflé’s work focuses on...
4,tmu,Scott Franks,Assistant Professor,scott.franks@torontomu.ca,https://www.torontomu.ca/law/faculty-and-resea...,Scott Franks is an assistant professor in the ...
5,tmu,Sari Graben,Associate Dean Research &amp; Graduate Studies...,sgraben@torontomu.ca,https://www.torontomu.ca/law/faculty-and-resea...,Sari Graben’s teaching and research focuses on...
6,tmu,Kathleen (Katie) Hammond,Assistant Professor,kathleen.hammond@torontomu.ca,https://www.torontomu.ca/law/faculty-and-resea...,Katie Hammond is an assistant professor in the...
7,tmu,Graham Hudson,"Associate Dean, Academic; Professor",graham.hudson@torontomu.ca,https://www.torontomu.ca/law/faculty-and-resea...,Graham Hudson is an Associate Professor and As...
8,tmu,Angela Lee,Assistant Professor,angela@torontomu.ca,https://www.torontomu.ca/law/faculty-and-resea...,Angela Lee joined the Lincoln Alexander School...
9,tmu,Avner Levin,Professor,avner.levin@torontomu.ca,https://www.torontomu.ca/law/faculty-and-resea...,Avner Levin is a professor at Toronto Metropol...


### Combine scraped data into a single file

In [9]:
# load Osgoode df
df = pd.read_json(json_outpath + 'osgoode_bios.json')

# add U of T df
tempdf = pd.read_json(json_outpath + 'u_toronto_bios.json')
df = pd.concat([df, tempdf], ignore_index=True)

# add TMU df
tempdf = pd.read_json(json_outpath + 'tmu_bios.json')
df = pd.concat([df, tempdf], ignore_index=True)

# Save to json for future use
df.to_json(json_outpath + 'all_bios.json', orient='records', indent = 2)
df

,faculty,name,title,email,href,bio
0,osgoode,Rabiat Akande,Assistant Professor,rakande@osgoode.yorku.ca,https://www.osgoode.yorku.ca/faculty-and-staff...,Professor Rabiat Akande works in the fields of...
1,osgoode,Harry Arthurs,Professor Emeritus,harthurs@osgoode.yorku.ca,https://www.osgoode.yorku.ca/faculty-and-staff...,"University Professor, former Dean of Osgoode H..."
2,osgoode,Saptarishi Bandopadhyay,Associate Professor,sbandopadhyay@osgoode.yorku.ca,https://www.osgoode.yorku.ca/faculty-and-staff...,I am an Associate Professor at Osgoode Hall La...
3,osgoode,Stephanie Ben-Ishai,Professor and York University Distinguished Re...,sbenishai@osgoode.yorku.ca,https://www.osgoode.yorku.ca/faculty-and-staff...,Professor Stephanie Ben-Ishai is a Distinguish...
4,osgoode,Benjamin L. Berger,Professor & York Research Chair in Pluralism a...,bberger@osgoode.yorku.ca,https://www.osgoode.yorku.ca/faculty-and-staff...,Professor Benjamin L. Berger is Professor and ...
...,...,...,...,...,...,...
156,tmu,Pnina Alon-Shenker,Associate Professor,pnina.alon@torontomu.ca,https://www.torontomu.ca/law/faculty-and-resea...,Pnina Alon-Shenker is an associate professor i...
157,tmu,Timothy J. Bartkiw,Associate Professor,tbartkiw@torontomu.ca,https://www.torontomu.ca/law/faculty-and-resea...,Timothy J. Bartkiw is an associate professor i...
158,tmu,Gil Lan,Associate Professor,glan@torontomu.ca,https://www.torontomu.ca/law/faculty-and-resea...,Gil Lan is an associate professor in the Linco...
159,tmu,Alexandra Orlova,Professor,aorlova@torontomu.ca,https://www.torontomu.ca/law/faculty-and-resea...,Alexandra Orlova received her PhD in law from ...
